In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install torch torchvision opencv-python tqdm lxml

In [ ]:
import os
import cv2
import torch
import numpy as np
import xml.etree.ElementTree as ET

from tqdm import tqdm
from torchvision.models import efficientnet_b0
import torch.nn as nn
import torchvision.transforms as transforms
from PIL import Image

In [ ]:
ROOT = "/content/drive/MyDrive/PCB_Dataset"

IMAGE_ROOT = ROOT + "/PCB_DATASET/images"
XML_ROOT   = ROOT + "/PCB_DATASET/Annotations"
TEMP_ROOT  = ROOT + "/PCB_DATASET/PCB_USED"

MODEL_PATH = ROOT + "/efficientnet_pcb.pth"

OUTPUT_DIR = ROOT + "/evaluation_outputs"

os.makedirs(OUTPUT_DIR, exist_ok=True)

In [ ]:
CLASSES = [
    "Missing_hole",
    "Mouse_bite",
    "Open_circuit",
    "Short",
    "Spur",
    "Spurious_copper"
]

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = efficientnet_b0()

model.classifier[1] = nn.Linear(
    model.classifier[1].in_features,
    len(CLASSES)
)

model.load_state_dict(torch.load(MODEL_PATH,map_location=device))

model.to(device)
model.eval()

EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActivat

In [ ]:
transform = transforms.Compose([

    transforms.Grayscale(3),
    transforms.Resize((224,224)),

    transforms.ToTensor(),

    transforms.Normalize(
        mean=[0.485,0.456,0.406],
        std =[0.229,0.224,0.225]
    )
])

In [ ]:
def detect_defects(template, test):

    template = cv2.resize(template,(test.shape[1],test.shape[0]))
    diff = cv2.absdiff(test,template)
    _, mask = cv2.threshold(
        diff,0,255,
        cv2.THRESH_BINARY+cv2.THRESH_OTSU
    )

    kernel = np.ones((3,3),np.uint8)

    mask = cv2.morphologyEx(mask,cv2.MORPH_OPEN,kernel)

    contours,_ = cv2.findContours(
        mask,
        cv2.RETR_EXTERNAL,
        cv2.CHAIN_APPROX_SIMPLE
    )

    rois = []
    boxes = []

    for c in contours:

        if cv2.contourArea(c)>250:

            x,y,w,h = cv2.boundingRect(c)
            roi = test[y:y+h,x:x+w]

            rois.append(roi)
            boxes.append((x,y,w,h))

    return rois,boxes

In [ ]:
def classify(model, rois, conf_thresh=0.2):

    preds = []
    scores = []

    for roi in rois:

        pil = Image.fromarray(roi)
        img = transform(pil).unsqueeze(0).to(device)

        with torch.no_grad():

            out = model(img)
            prob = torch.softmax(out, dim=1)

        conf, pred = torch.max(prob, 1)

        conf = conf.item()
        pred = pred.item()

        # Reject low-confidence predictions
        if conf > conf_thresh:
            preds.append(CLASSES[pred])
            scores.append(conf)

    return preds, scores

In [ ]:
def read_xml(xml_path):

    tree = ET.parse(xml_path)
    root = tree.getroot()

    objects = []

    for obj in root.findall("object"):

        name = obj.find("name").text

        box = obj.find("bndbox")

        xmin = int(box.find("xmin").text)
        ymin = int(box.find("ymin").text)
        xmax = int(box.find("xmax").text)
        ymax = int(box.find("ymax").text)

        objects.append((name,(xmin,ymin,xmax,ymax)))

    return objects

In [ ]:
def iou(a,b):

    xA = max(a[0],b[0])
    yA = max(a[1],b[1])
    xB = min(a[2],b[2])
    yB = min(a[3],b[3])

    inter = max(0,xB-xA)*max(0,yB-yA)

    areaA = (a[2]-a[0])*(a[3]-a[1])
    areaB = (b[2]-b[0])*(b[3]-b[1])

    union = areaA+areaB-inter

    return inter/union if union>0 else 0

In [ ]:
def nms(boxes, scores, iou_thresh=0.2):

    if len(boxes) == 0:
        return []

    boxes = np.array(boxes)
    scores = np.array(scores)

    x1 = boxes[:,0]
    y1 = boxes[:,1]
    x2 = boxes[:,0] + boxes[:,2]
    y2 = boxes[:,1] + boxes[:,3]
    areas = (x2-x1)*(y2-y1)
    order = scores.argsort()[::-1]
    keep = []

    while order.size > 0:
        i = order[0]
        keep.append(i)

        xx1 = np.maximum(x1[i], x1[order[1:]])
        yy1 = np.maximum(y1[i], y1[order[1:]])
        xx2 = np.minimum(x2[i], x2[order[1:]])
        yy2 = np.minimum(y2[i], y2[order[1:]])

        w = np.maximum(0, xx2-xx1)
        h = np.maximum(0, yy2-yy1)
        inter = w*h
        ovr = inter / (areas[i] + areas[order[1:]] - inter)
        inds = np.where(ovr <= iou_thresh)[0]
        order = order[inds+1]
    return keep

In [ ]:
TP=FP=FN=0

results = []


for cls in os.listdir(XML_ROOT):

    xml_dir = os.path.join(XML_ROOT,cls)
    img_dir = os.path.join(IMAGE_ROOT,cls)

    for xml in tqdm(os.listdir(xml_dir)):

        xml_path = os.path.join(xml_dir,xml)

        gt = read_xml(xml_path)

        fname = ET.parse(xml_path).getroot().find("filename").text

        img_path = os.path.join(img_dir,fname)


        pcb_id = fname.split("_")[0]

        temp_path = os.path.join(TEMP_ROOT,pcb_id+".JPG")

        if not os.path.exists(temp_path):
            continue


        template = cv2.imread(temp_path,0)
        test = cv2.imread(img_path,0)


        rois,boxes = detect_defects(template,test)

        preds, scores = classify(model, rois, conf_thresh=0.2)

        keep = nms(boxes, scores, iou_thresh=0.2)
        boxes = [boxes[i] for i in keep]
        preds = [preds[i] for i in keep]
        scores = [scores[i] for i in keep]

        def normalize(x):
            return x.lower().replace("-", "_").strip()
        used_gt = [False] * len(gt)

        # For each prediction
        for (x,y,w,h), pred_cls in zip(boxes, preds):
            pred_box = (x, y, x+w, y+h)
            best_iou = 0
            best_gt  = -1

            # Find best matching GT box
            for i, (gt_cls, gt_box) in enumerate(gt):

                if used_gt[i]:
                    continue
                overlap = iou(pred_box, gt_box)

                if overlap > best_iou:
                    best_iou = overlap
                    best_gt = i

            # If good match found
            if best_iou > 0.1:
                gt_cls, _ = gt[best_gt]
                if normalize(pred_cls) == normalize(gt_cls):
                    TP += 1
                else:
                    FP += 1
                used_gt[best_gt] = True
            # No match → false positive
            else:
                FP += 1
        # Count missed GT
        FN += sum([not x for x in used_gt])

        # Save visualization
        vis = cv2.cvtColor(test,cv2.COLOR_GRAY2BGR)

        for (x,y,w,h),p in zip(boxes,preds):

            cv2.rectangle(vis,(x,y),(x+w,y+h),(0,255,0),2)
            cv2.putText(vis,p,(x,y-5),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        0.6,(0,0,255),2)

        cv2.imwrite(
            os.path.join(OUTPUT_DIR,fname),
            vis
        )

100%|██████████| 115/115 [00:52<00:00,  2.17it/s]


In [ ]:
precision = TP/(TP+FP+1e-6)
recall    = TP/(TP+FN+1e-6)
f1        = 2*precision*recall/(precision+recall+1e-6)

print("TP:",TP)
print("FP:",FP)
print("FN:",FN)

print("Precision:",precision)
print("Recall:",recall)
print("F1 Score:",f1)

match_rate = TP/(TP+FN+1e-6)

print("Prediction Match Rate:",match_rate)

TP: 2315
FP: 2248
FN: 408
Precision: 0.507341661076629
Recall: 0.8501652585934024
F1 Score: 0.6354648075854472
Prediction Match Rate: 0.8501652585934024


In [ ]:
debug_img = cv2.cvtColor(test,cv2.COLOR_GRAY2BGR)

# Draw GT (Blue)
for cls,box in gt:
    x1,y1,x2,y2 = box
    cv2.rectangle(debug_img,(x1,y1),(x2,y2),(255,0,0),2)

# Draw Pred (Green)
for (x,y,w,h),p in zip(boxes,preds):
    cv2.rectangle(debug_img,(x,y),(x+w,y+h),(0,255,0),2)

cv2.imwrite("debug.jpg",debug_img)

True